In [ ]:
import pandas as pd

import numpy as np
import gc


In [ ]:
# Read datasets back in cleanly
df_train = pd.read_pickle('/kaggle/input/processedata/traindata5.pkl')
df_test = pd.read_pickle('/kaggle/input/processedata/testdata5.pkl')
sequence_length=5
df_train.head(sequence_length)
#df_train.fillna(0)
#df_test.fillna(0)

In [ ]:
df_train = df_train.dropna(axis='columns', how='all') 
# Next, let's remove a few rows that had invalid data (most likely error reading SMART statistics)
df_train = df_train.dropna(axis='rows', how='any')
df_test = df_test.dropna(axis='columns', how='all') 
df_test = df_test.dropna(axis='rows', how='any')
'''df_train.dropna(how='all', axis=1, inplace=True)
df_test.dropna(how='all', axis=1, inplace=True)
df_train = df_train.dropna(axis='rows', how='any')
df_test = df_test.dropna(axis='rows', how='any')'''


#col_list_raw = remove_constant_values(df_failed,"_raw")
#col_list_normalized = remove_constant_values(df_failed,"_normalized")
#col_list = ['date', 'serial_number', 'failure'] 
#col_list = col_list + col_list_raw + col_list_normalized

#df_train = df_train[col_list]

#keep the same columns as the training dataframe 
#df_test = df_test[col_list]
df_test = df_test.reindex(columns=df_train.columns)

#fill in the small number of null values in remaining columns 
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [ ]:
# index data will hold disk serial numbers for relevant samples. We do not need serial numbers as training or testing features
drop_columns_list = ['date','serial_number', 'failure', 'sequence_label']

x_train = df_train.drop(columns=drop_columns_list)
x_test = df_test.drop(columns=drop_columns_list)

y_train = df_train['sequence_label']
y_test = df_test['sequence_label']

In [ ]:

from sklearn import preprocessing


x_train = x_train.values.astype(np.float64)



# X test data
x_test = x_test.values.astype(np.float64)


import pandas as pd

# Convert numpy arrays to pandas dataframes
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)

# Filter out extra columns in the training data
cols_to_drop = set(x_train.columns) - set(x_test.columns)
x_train = x_train.drop(cols_to_drop, axis=1)

# Add missing columns in the testing data
missing_cols = set(x_train.columns) - set(x_test.columns)
for col in missing_cols:
    x_test[col] = 0

# Apply scaler
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)



In [ ]:
x_test

In [ ]:
print("Reshaping training and test data for LSTM")
print("- Original Train Shape:", x_train.shape)

x_train = x_train.reshape(int(x_train.shape[0]/sequence_length), sequence_length, x_train.shape[1])
x_test = x_test.reshape(int(x_test.shape[0]/sequence_length), sequence_length, x_test.shape[1])

print("- Final Train Shape:   ", x_train.shape)
data_shape = x_train.shape[1:]

In [ ]:
# Y train data
y_train = y_train.values[0::sequence_length]

# Y test data
y_test = y_test.values[0::sequence_length]

gc.collect()

In [ ]:
import keras.backend as K

def matthews_correlation(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1-y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1-y_pred), 0, 1)))

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [ ]:
nb_features = x_train.shape[2]
timestamp = 5
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras import regularizers
from keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adagrad
from keras import backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras import backend as K
from keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


dp_lvl = 0.2
regularizer_lvl = 0.002
units = 64

epochs_num = 100
learning_rate = 0.001
decay_rate = 3 * learning_rate / epochs_num
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None,decay = decay_rate)
#optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

model = Sequential()
model.add(LSTM(units, input_shape=(x_train.shape[1], x_train.shape[2]),dropout = dp_lvl,recurrent_dropout = dp_lvl, return_sequences =  True ))
model.add(LSTM(units, dropout = dp_lvl,recurrent_dropout = dp_lvl, return_sequences =  True ))
model.add(LSTM(units, dropout = dp_lvl,recurrent_dropout = dp_lvl, return_sequences =  False ))
model.add(Dense(units, activation='tanh',activity_regularizer=regularizers.l2(regularizer_lvl)))

    # Dropout is a technique used to tackle Overfitting.
model.add(Dropout (0.2))
model.add(Dense(units, activation='tanh',activity_regularizer=regularizers.l2(regularizer_lvl)))
model.add(Dense(1, activation='tanh',activity_regularizer=regularizers.l2(regularizer_lvl)))
#model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy',metrics.Recall(),matthews_correlation])
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=[metrics.Recall(),'accuracy'])
model_fn = "best_model.h5"
model_filepath=model_fn

checkpoint = ModelCheckpoint(model_filepath, monitor='val_recall', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(patience=20, verbose=1)

callbacks_list = [es, checkpoint]
# Compute the class weights


# Train the model on the entire training set with class weights and early stopping
#history = model.fit(x_train, y_train, epochs=100, batch_size=16,  callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
                       #])
history = model.fit(x_train, y_train, epochs=epochs_num, batch_size=16, validation_split=0.1, verbose=1,  shuffle=True,class_weight=class_weights_dict, callbacks=callbacks_list)
# Print the final training accuracy, validation accuracy, and recall
#print("Final training accuracy:", history.history['accuracy'])


In [ ]:
y_pred = model.predict(x_test)

# convert probabilities to binary predictions (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_binary)
print(cm)
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy:", accuracy)

In [ ]:
from keras.models import load_model

model = load_model('best_model.h5')


In [ ]:
dp_lvl = 0.2
regularizer_lvl = 0.001
units = 64

epochs_num = 50
learning_rate = 0.001
decay_rate = 3 * learning_rate / epochs_num
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate)

new_layers = [
    LSTM(units, input_shape=(x_train.shape[1], x_train.shape[2]), dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=True),
    LSTM(units, dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=True),
    LSTM(units, dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=False),
    Dense(units, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl)),
    Dropout(0.2),
    Dense(units, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl)),
    Dense(1, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl))
]

# Replace the existing layers with the new ones
for i, layer in enumerate(new_layers):
    model.layers[i] = layer

# Compile the modified model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics.Recall(), 'accuracy'])


In [ ]:
history = model.fit(x_train, y_train, epochs=epochs_num, batch_size=16, validation_split=0.1, verbose=1,  shuffle=True, class_weight=class_weights_dict, callbacks=callbacks_list)


In [ ]:
# load weights from best model
#model.load_weights(model_filepath)

# evaluate model against test set
#model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy', metrics.Recall()]) 
scores = model.evaluate(x_test, y_test, verbose=1)
print("%s: %.2f%% %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))

In [ ]:
dp_lvl = 0.2
regularizer_lvl = 0.002
units = 64

epochs_num = 50
learning_rate = 0.0001
decay_rate = 3 * learning_rate / epochs_num
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate)

new_layers = [
    LSTM(units, input_shape=(x_train.shape[1], x_train.shape[2]), dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=True),
    LSTM(units, dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=True),
    LSTM(units, dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=False),
    Dense(units, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl)),
    Dropout(0.2),
    Dense(units, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl)),
    Dense(1, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl))
]

# Replace the existing layers with the new ones
for i, layer in enumerate(new_layers):
    model.layers[i] = layer

# Compile the modified model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[ 'accuracy',metrics.Recall()])


In [ ]:
history = model.fit(x_train, y_train, epochs=epochs_num, batch_size=16, validation_split=0.1, verbose=1,  shuffle=True, class_weight=class_weights_dict, callbacks=callbacks_list)


In [ ]:
y_pred = model.predict(x_test)

# convert probabilities to binary predictions (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_binary)
print(cm)
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy:", accuracy)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("%s: %.2f%% %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))

In [ ]:
dp_lvl = 0.2
regularizer_lvl = 0.001
units = 64

epochs_num = 50
learning_rate = 0.0001
decay_rate = 3 * learning_rate / epochs_num
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate)

new_layers = [
    LSTM(units, input_shape=(x_train.shape[1], x_train.shape[2]), dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=True),
    LSTM(units, dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=True),
    LSTM(units, dropout=dp_lvl, recurrent_dropout=dp_lvl, return_sequences=False),
    Dense(units, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl)),
    Dropout(0.2),
    Dense(units, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl)),
    Dense(1, activation='tanh', activity_regularizer=regularizers.l2(regularizer_lvl))
]

# Replace the existing layers with the new ones
for i, layer in enumerate(new_layers):
    model.layers[i] = layer

# Compile the modified model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics.Recall(),'accuracy'])


In [ ]:
history = model.fit(x_train, y_train, epochs=epochs_num, batch_size=16, validation_split=0.1, verbose=1,  shuffle=True, class_weight=class_weights_dict, callbacks=callbacks_list)


In [ ]:
y_pred = model.predict(x_test)

# convert probabilities to binary predictions (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# compute confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_binary)
print(cm)
TP = cm[0][0]
TN = cm[1][1]
FP = cm[0][1]
FN = cm[1][0]
accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy:", accuracy)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("%s: %.2f%% %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))

In [ ]:
model.save('hi.h5')